# Rule Based Machine Learning Algorithm

In [1]:
filename = 'user-ct-test-collection-01.txt'

In [2]:
# !rm -rf user-ct-test-collection-01.txt

## Get Dataset

In [3]:
import os
import gzip
import ssl
import shutil
import urllib2

if os.path.isfile(filename):
    print 'Dataset found'
else:
    print 'Downloading dataset'
    f = urllib2.urlopen('https://github.com/dimosr/Big_Data/raw/master/resources/user-ct-test-collection-01.txt.gz', 
                        context=ssl.SSLContext(ssl.PROTOCOL_TLS))
    with open(os.path.basename(filename + '.gz'), 'wb') as local_file:
        local_file.write(f.read())
    print 'Download complete, unzipping'
    with gzip.open(filename + '.gz', 'rb') as f_in:
        with open(filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove(filename + '.gz')        
    print 'Unzip complete'

Download complete, unzipping
Unzip complete


## Preparing the Dataset

In [8]:
from pyspark.sql.functions import col

rdd = sc.textFile(filename).map(lambda x: x.split('\t'))
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)
df = rdd.toDF(header)
df.show(100)

+------+--------------------+-------------------+--------+--------------------+
|AnonID|               Query|          QueryTime|ItemRank|            ClickURL|
+------+--------------------+-------------------+--------+--------------------+
|   142|      rentdirect.com|2006-03-01 07:17:12|        |                    |
|   142|www.prescriptionf...|2006-03-12 12:31:06|        |                    |
|   142|          staple.com|2006-03-17 21:19:29|        |                    |
|   142|          staple.com|2006-03-17 21:19:45|        |                    |
|   142|www.newyorklawyer...|2006-03-18 08:02:58|        |                    |
|   142|www.newyorklawyer...|2006-03-18 08:03:09|        |                    |
|   142|     westchester.gov|2006-03-20 03:55:57|       1|http://www.westch...|
|   142|       space.comhttp|2006-03-24 20:51:24|        |                    |
|   142|                dfdf|2006-03-24 22:23:07|        |                    |
|   142|                dfdf|2006-03-24 

## Preprocessing

In [45]:
from pyspark.sql.functions import *

df.createOrReplaceTempView('df')
df = spark.sql("SELECT AnonID, Query, QueryTime FROM df")

df = df.withColumn('Query', regexp_replace('Query', 'www.|.html|.com|.gov|.org|.co|.net|.edu|.mil', ''))
# df = df.filter(size(df.Query) > 1)

df.show(500)

KeyboardInterrupt: 

In [6]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import count, col

# minutes = lambda i: i * 60 

# w = (Window()
#    .partitionBy(col('AnonID'))
#    .orderBy(col('QueryTime').cast('timestamp').cast('long'))
#    .rangeBetween(-minutes(10), 0))

# df.select(col('*'), count('Query').over(w).alias('mean')).show()

+--------+--------------------+-------------------+----+
|  AnonID|               Query|          QueryTime|mean|
+--------+--------------------+-------------------+----+
|10086493|       tiffany's.com|2006-03-06 16:11:06|   1|
|10086493|n4514524-452881-4...|2006-03-08 01:06:16|   1|
|10086493|          sprint pcs|2006-03-20 01:05:26|   1|
|10086493|             atacand|2006-03-20 01:35:36|   1|
|10086493|    jennifer nettles|2006-03-20 01:48:56|   1|
|10086493|    jennifer nettles|2006-03-20 01:49:03|   3|
|10086493|    jennifer nettles|2006-03-20 01:49:03|   3|
|10086493|    jennifer nettles|2006-03-20 01:49:19|   4|
|10086493|    jennifer nettles|2006-03-20 01:49:34|   5|
|10086493|    jennifer nettles|2006-03-20 01:51:25|   7|
|10086493|    jennifer nettles|2006-03-20 01:51:25|   7|
|10086493|olivegarden las v...|2006-03-21 22:50:38|   2|
|10086493|olivegarden las v...|2006-03-21 22:50:38|   2|
|10086493|olivegarden las v...|2006-03-21 22:52:34|   3|
|10086493|olive garden las ...|